In [1]:
import os
import sys
import math
import psutil
import sklearn
import humanize
import warnings
import subprocess
import numpy as np
import pandas as pd
import GPUtil as GPU
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.python.client import device_lib

In [2]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [3]:
import keras
import keras.backend as K
from keras.utils import to_categorical
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint
from keras.initializers import glorot_uniform
from keras.layers.core import Permute, Reshape, RepeatVector
from keras.losses import cosine_proximity, categorical_crossentropy
from keras.layers import Input, Dense, Dropout, CuDNNGRU, Embedding, concatenate, Lambda, multiply, merge, Flatten

Using TensorFlow backend.


In [4]:
class SessionDataset:
    """Credit to yhs-968/pyGRU4REC."""    
    def __init__(self, data, sep='\t', user_key='UserId', movie_key='MovieId', time_key='Time', n_samples=-1, movie_map=None, time_sort=False):
        """
        Args:
            data: 讀取好的dataframe
            sep: separator for the csv
            user_key, movie_key, time_key: name of the fields corresponding to the users, movies, time
            n_samples: the number of samples to use. If -1, use the whole dataset.
            movie_map: mapping between movie IDs and movie indices
            time_sort: whether to sort the users by time or not
        """
        self.df = data
        self.user_key = user_key
        self.movie_key = movie_key
        self.time_sort = time_sort #False
        self.add_item_indices(movie_map=movie_map)
        self.df.sort_values([user_key, time_key], inplace=True)

        #Sort the df by time, and then by session ID. That is, df is sorted by session ID and
        #clicks within a session are next to each other, where the clicks within a session are time-ordered.

        self.click_offsets = self.get_click_offsets()
        self.session_idx_arr = self.order_session_idx()
        
    def get_click_offsets(self):
        """
        Return the offsets of the beginning clicks of each session IDs,
        where the offset is calculated against the first click of the first session ID.
        """
        offsets = np.zeros(self.df[self.user_key].nunique() + 1, dtype=np.int32)
        # group & sort the df by user_key and get the offset values
        offsets[1:] = self.df.groupby(self.user_key).size().cumsum()

        return offsets

    def order_session_idx(self):
        """ Order the session indices """
        if self.time_sort:
            # starting time for each users, sorted by session IDs
            users_start_time = self.df.groupby(self.user_key)[self.time_key].min().values
            # order the session indices by session starting times
            session_idx_arr = np.argsort(users_start_time)
        else:
            session_idx_arr = np.arange(self.df[self.user_key].nunique())

        return session_idx_arr
    
    def add_item_indices(self, movie_map=None):
        """ 
        Add item index column named "item_idx" to the df
        Args:
            movie_map (pd.DataFrame): mapping between the item Ids and indices
        """
        if movie_map is None:
            item_ids = self.df[self.movie_key].unique()  # unique item ids
            item2idx = pd.Series(data=np.arange(len(item_ids)),
                                 index=item_ids)
            movie_map = pd.DataFrame({self.movie_key:item_ids,
                                   'item_idx':item2idx[item_ids].values})
        
        self.movie_map = movie_map
        self.df = pd.merge(self.df, self.movie_map, on=self.movie_key, how='inner')
        
    @property    
    def items(self):
        return self.movie_map.MovieId.unique()

In [5]:
def create_model():   
    emb_size = 50
    hidden_units = 100
    size = emb_size

    inputs = Input(batch_shape=(batch_size, 1, num_movies))
    gru, gru_states = CuDNNGRU(hidden_units, stateful=True, return_state=True)(inputs)# drop1) #
    drop2 = Dropout(0.25)(gru)
    predictions = Dense(num_movies, activation='softmax')(drop2)
    
    model = Model(input=inputs, output=[predictions])
    
    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss=categorical_crossentropy, optimizer=opt)
    model.summary()

    filepath='DwellTimeModel_checkpoint.h5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=2, save_best_only=True, mode='min')
    callbacks_list = []
    return model


def get_states(model):
    return [K.get_value(s) for s,_ in model.state_updates]


def set_states(model, states):
    for (d,_), s in zip(model.state_updates, states):
        K.set_value(d, s)


def get_recall(model, loader, epoch, train_generator_map, recall_k=20):

    test_dataset = SessionDataset(test_data, movie_map=train_generator_map)
    test_generator = SessionDataLoader(test_dataset, batch_size=batch_size)

    n = 0
    suma = 0
    suma_baseline = 0

    for feat, label, mask in test_generator:

        input_oh = to_categorical(feat, num_classes=loader.num_movies) 
        input_oh = np.expand_dims(input_oh, axis=1)
        target_oh = to_categorical(label, num_classes=loader.num_movies)
        pred = model.predict(input_oh, batch_size=batch_size)

        if n%100 == 0:
            try:
                print("{}:{}".format(n, suma/n))
            except:
                pass

        for row_idx in range(feat.shape[0]):
            pred_row = pred[row_idx] 
            label_row = target_oh[row_idx]

            idx1 = pred_row.argsort()[-recall_k:][::-1]
            idx2 = label_row.argsort()[-1:][::-1]

            n += 1
            if idx2[0] in idx1:
                suma += 1

    print("Recall@{} epoch {}: {}".format(recall_k, epoch, suma/n))


def get_mrr(model, loader,epoch,train_generator_map, mrr_k=20):

    test_dataset = SessionDataset(test_data, movie_map = train_generator_map)
    test_generator = SessionDataLoader(test_dataset, batch_size=batch_size)

    n = 0
    suma = 0
    suma_baseline = 0

    for feat, label, mask in test_generator:
        input_oh = to_categorical(feat, num_classes=loader.num_movies) 
        input_oh = np.expand_dims(input_oh, axis=1)
        target_oh = to_categorical(label, num_classes=loader.num_movies)
        pred = model.predict(input_oh, batch_size=batch_size)

        if n%100 == 0:
            try:
                print("{}:{}".format(n, suma/n))
            except:
                pass

        for row_idx in range(feat.shape[0]):
            pred_row = pred[row_idx] 
            label_row = target_oh[row_idx]

            idx1 = pred_row.argsort()[-mrr_k:][::-1]
            idx2 = label_row.argsort()[-1:][::-1]

            n += 1
            if idx2[0] in idx1:
                suma += 1/int((np.where(idx1 == idx2[0])[0]+1))        

    print("MRR@{} epoch {}: {}".format(mrr_k, epoch, suma/n))


def train_model(model, save_weights = False, path_to_weights = True):
    train_dataset = SessionDataset(train_data)
    
    print("train_dataset \n", train_dataset)

    model_to_train = model

    with tqdm(total=train_samples_qty) as pbar: #顯示處理進度條，每一個user處理一次
        for epoch in range(1, 10):  #跑9次
            
            if path_to_weights: # 
                loader = SessionDataLoader(train_dataset, batch_size=batch_size)
                
                
            for feat, target, mask in loader:
                #print("feat", feat)
                #print("target", target)
                #print("mask", mask)

                input_oh = to_categorical(feat, num_classes=loader.num_movies) 
                input_oh = np.expand_dims(input_oh, axis=1)
                #print("input_oh", input_oh)

                target_oh = to_categorical(target, num_classes=loader.num_movies)
                #print("target_oh", target_oh)
                
                tr_loss = model_to_train.train_on_batch(input_oh, target_oh)
                #print("tr_loss", tr_loss)

                real_mask = np.ones((batch_size, 1))
                for elt in mask:
                    real_mask[elt, :] = 0

                hidden_states = get_states(model_to_train)[0]
                hidden_states = np.multiply(real_mask, hidden_states)
                hidden_states = np.array(hidden_states, dtype=np.float32)
                #print("hidden_states", hidden_states)
                
                model_to_train.layers[1].reset_states(hidden_states)

                pbar.set_description("Epoch {0}. Loss: {1:.5f}".format(epoch, tr_loss))
                pbar.update(loader.done_users_counter)

            # get metrics for epoch
            get_recall(model_to_train, loader, epoch, train_dataset.movie_map)
            get_mrr   (model_to_train, loader, epoch, train_dataset.movie_map)

            # save model
            if save_weights:
                model_to_train.save('DwellTimeEpoch{}.h5'.format(epoch))

In [6]:
class SessionDataLoader:
    """Credit to yhs-968/pyGRU4REC."""    
    def __init__(self, dataset, batch_size=50):
        """
        A class for creating session-parallel mini-batches.
        Args:
             dataset (SessionDataset): the session dataset to generate the batches from
             batch_size (int): size of the batch
        """
        self.dataset = dataset
        self.batch_size = batch_size
        self.done_users_counter = 0
        
    def __iter__(self):
        """ Returns the iterator for producing session-parallel training mini-batches.
        Yields:
            input (B,):  Item indices that will be encoded as one-hot vectors later.
            target (B,): a Variable that stores the target item indices
            masks: Numpy array indicating the positions of the users to be terminated
        """

        # initializations
        df = self.dataset.df
        user_key='UserId'
        movie_key='MovieId'
        time_key='TimeStamp'
        self.num_movies = df[movie_key].nunique()+1
        click_offsets = self.dataset.click_offsets
        session_idx_arr = self.dataset.session_idx_arr

        iters = np.arange(self.batch_size)
        maxiter = iters.max()
        start = click_offsets[session_idx_arr[iters]]
        end = click_offsets[session_idx_arr[iters] + 1]
        mask = [] # indicator for the users to be terminated
        finished = False        
        
        

        while not finished:
            minlen = (end - start).min()
            # Item indices (for embedding) for clicks where the first users start
            idx_target = df.item_idx.values[start]
            for i in range(minlen - 1):
                # Build inputs & targets
                idx_input = idx_target
                idx_target = df.item_idx.values[start + i + 1]
                input = idx_input
                target = idx_target
                yield input, target, mask
                
            # click indices where a particular session meets second-to-last element
            start = start + (minlen - 1)
            # see if how many users should terminate
            mask = np.arange(len(iters))[(end - start) <= 1]
            self.done_users_counter = len(mask)
            for idx in mask:
                maxiter += 1
                if maxiter >= len(click_offsets) - 1:
                    finished = True
                    break
                # update the next starting/ending point
                iters[idx] = maxiter
                start[idx] = click_offsets[session_idx_arr[maxiter]]
                end[idx] = click_offsets[session_idx_arr[maxiter] + 1]
        
        #print("num_movies ",self.num_movies)
        #print("click_offsets ",click_offsets)
        #print("session_idx_arr ",session_idx_arr)
        #print("iters ",iters)
        #print("maxiter " ,maxiter)
        #print("start ",start)
        #print("end ",end)
        #print("mask ",mask)
        #print("finished ",finished)

In [7]:
    PATH_TO_TRAIN = 'processed_augmented_train.csv'
    PATH_TO_DEV = 'processed_dev.csv'
    PATH_TO_TEST = 'processed_test.csv'
    train_data = pd.read_csv(PATH_TO_TRAIN, sep='\t', dtype={'MovieId':np.int64})
    dev_data = pd.read_csv(PATH_TO_DEV, sep='\t', dtype={'MovieId':np.int64})
    test_data = pd.read_csv(PATH_TO_TEST, sep='\t', dtype={'MovieId': np.int64})
    
    batch_size = 512
    session_max_len = 100
    embeddingp=False

    num_movies = len(train_data['MovieId'].unique())+1
    print("Unique training movies:", num_movies)

    dev_num_movies = len(dev_data['MovieId'].unique())+1
    print("Unique dev movies:", dev_num_movies)

    test_num_movies = len(test_data['MovieId'].unique())+1
    print("Unique testing movies:", test_num_movies)

    train_samples_qty = len(train_data['UserId'].unique()) 
    print("Training users:", train_samples_qty)

    dev_samples_qty = len(dev_data['UserId'].unique()) 
    print("Dev users:",dev_samples_qty)

    test_samples_qty = len(test_data['UserId'].unique())
    print("Testing users:", test_samples_qty)
    
    train_fraction = 1 # (1 / fraction) most recent session quantity to consider
    dev_fraction = 1

    train_offset_step=train_samples_qty//batch_size
    dev_offset_step=dev_samples_qty//batch_size
    test_offset_step=test_samples_qty//batch_size
    aux = [0]
    aux.extend(list(train_data['MovieId'].unique()))
    itemids = np.array(aux)
    itemidmap = pd.Series(data=np.arange(num_movies), index=itemids) 
    
    model = create_model()
    
    train_model(model)

Unique training movies: 11619
Unique dev movies: 10105
Unique testing movies: 10366
Training users: 19853
Dev users: 5749
Testing users: 5271
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (512, 1, 11619)           0         
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       [(512, 100), (512, 100)]  3516300   
_________________________________________________________________
dropout_1 (Dropout)          (512, 100)                0         
_________________________________________________________________
dense_1 (Dense)              (512, 11619)              1173519   
Total params: 4,689,819
Trainable params: 4,689,819
Non-trainable params: 0
_

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=Tensor("in...)`
  # This is added back by InteractiveShellApp.init_path()
  0%|          | 0/19853 [00:00<?, ?it/s]

train_dataset 
Instructions for updating:
Use tf.cast instead.


Epoch 1. Loss: 5.00708:  98%|█████████▊| 19408/19853 [04:02<00:04, 89.60it/s] 

12800:0.100234375


...

12800:0.138203125


Epoch 9. Loss: 3.31624: : 174735it [1:01:31, 115.02it/s]

25600:0.13390625
38400:0.13192708333333333
51200:0.130625
64000:0.129328125
76800:0.12954427083333334
89600:0.12982142857142856
102400:0.13138671875
115200:0.1321875
128000:0.1322421875
140800:0.1320028409090909
153600:0.13239583333333332
Recall@20 epoch 9: 0.1324585355987055
12800:0.028832712789245023
25600:0.028095617603749044
38400:0.02729182868266846
51200:0.027318726554686236
64000:0.02680505349000359
76800:0.026604746100969072
89600:0.026660255416019237
102400:0.027056620328798235
115200:0.027245063146182247
128000:0.02719429955951851
140800:0.027215120384645134
153600:0.027209141745917154


Epoch 9. Loss: 3.31624: : 174735it [1:05:42, 44.33it/s] 

MRR@20 epoch 9: 0.027240993072724835
